In [121]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
df = df.drop(columns=['volume','change', 'low', 'high', 'open'])
df = df.rename(columns={"value": "wallets"})
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
df.dtypes

#https://dcresearch.medium.com/metcalfes-law-and-bitcoin-s-value-2b99c7efd1fa

date       datetime64[ns]
price             float64
wallets             int64
address           float64
mined             float64
dtype: object

In [122]:
df['Metcafe']=df['address']**2
df['value'] = df['Metcafe']/df['mined']
df["value"] = df["value"].map("{:.2f}".format)
df['value']=df['value'].astype("float")
df['networkvalue'] = df["price"] - df["value"]
df.dtypes


date            datetime64[ns]
price                  float64
wallets                  int64
address                float64
mined                  float64
Metcafe                float64
value                  float64
networkvalue           float64
dtype: object

In [123]:
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df['400D'] = df['price'].rolling(400).mean()

df = df.dropna()

In [133]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(name="Actual", x=df['date'], y=df['price'], legendrank=4))
fig.add_trace(go.Scatter(name="200 Day", x=df['date'], y=df['200D'], legendrank=2))
fig.add_trace(go.Scatter(name="300 Day", x=df['date'], y=df['300D'], legendrank=2))
fig.add_trace(go.Scatter(name="50 Day", x=df['date'], y=df['50D'], legendrank=2))
fig.add_trace(go.Scatter(name="400 Day", x=df['date'], y=df['400D'], legendrank=2))

fig.update_xaxes(
    rangeslider_visible = True,
    rangeselector = dict(
        buttons = list([
            dict(count = 1, label = "1m", step = "month", stepmode = "backward"),
            dict(count = 6, label = "6m", step = "month", stepmode = "backward"),
            dict(count = 1, label = "YTD", step = "year", stepmode = "todate"),
            dict(count = 1, label = "1y", step = "year", stepmode = "backward"),
            dict(step = "all")
        ])
    )
)

fig.update_layout(title_text='Bitcoin Wallets and Date Prediction')
fig.update_yaxes(ticklabelposition="inside top", title="Bitcoin Wallets")
fig.update_xaxes(ticklabelposition="inside top", title="Date")
fig.update_yaxes(nticks=15)
fig.update_xaxes(nticks=20)
fig.update_layout(
    margin=dict(l=20, r=100, t=70, b=20),
)
fig.show()




In [134]:
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] + df['400D'])/4
df

,date,price,wallets,address,mined,Metcafe,value,networkvalue,200D,300D,50D,400D,meanavge
399,2011-10-29,3.6,2,12691.0,7557550.00,1.610615e+08,21.31,-17.71,8.9605,6.214000,4.350,4.71100,6.058875
400,2011-10-30,3.3,2,11261.0,7564400.00,1.268101e+08,16.76,-13.46,8.9725,6.224000,4.320,4.71900,6.058875
401,2011-10-31,3.3,2,11738.0,7559300.00,1.377806e+08,18.23,-14.93,8.9840,6.234000,4.268,4.72700,6.053250
402,2011-11-01,3.2,2,12519.0,7579250.00,1.567254e+08,20.68,-17.48,8.9950,6.243667,4.210,4.73475,6.045854
403,2011-11-02,3.3,2,12907.0,7585600.00,1.665906e+08,21.96,-18.66,9.0065,6.253667,4.160,4.74275,6.040729
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,2022-10-21,19162.6,84879615,924681.0,19185681.25,8.550350e+11,44566.31,-25403.71,25662.9880,31072.760000,19619.662,36971.18700,28331.649250
4382,2022-10-22,19204.8,84888147,830404.0,19186618.75,6.895708e+11,35940.19,-16735.39,25531.4820,30967.509667,19604.704,36900.99200,28251.171917
4383,2022-10-23,19571.2,84893809,804140.0,19187600.00,6.466411e+11,33700.99,-14129.79,25413.4730,30863.819667,19599.500,36829.15325,28176.486479
4384,2022-10-24,19331.5,84902680,919344.0,19188556.25,8.451934e+11,44046.74,-24715.24,25292.8905,30769.774000,19586.132,36759.38525,28102.045438


In [142]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(name="Actual", x=df['date'], y=df['price'], legendrank=4))
fig.add_trace(go.Scatter(name="MeanAvg", x=df['date'], y=df['meanavge'], legendrank=2))
fig.add_trace(go.Scatter(name="value", x=df['date'], y=df['value'], legendrank=2))


fig.update_xaxes(
    rangeslider_visible = True,
    rangeselector = dict(
        buttons = list([
            dict(count = 1, label = "1m", step = "month", stepmode = "backward"),
            dict(count = 6, label = "6m", step = "month", stepmode = "backward"),
            dict(count = 1, label = "YTD", step = "year", stepmode = "todate"),
            dict(count = 1, label = "1y", step = "year", stepmode = "backward"),
            dict(step = "all")
        ])
    )
)
fig.update_layout(title_text='Bitcoin Wallets and Date Prediction')
fig.update_yaxes(ticklabelposition="inside top", title="Bitcoin Wallets")
fig.update_xaxes(ticklabelposition="inside top", title="Date")
fig.update_yaxes(nticks=15)
fig.update_xaxes(nticks=20)
fig.update_layout(
    margin=dict(l=20, r=100, t=70, b=20),
)
fig.show()

In [ ]:

# Create our features
X = df.drop(columns="status")
X = pd.get_dummies(X)

# Create our target
X = df.drop(columns="date")
y = df['status']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, train_size=0.70)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

In [ ]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
LRclf = LogisticRegression().fit(X_train, y_train)
y_pred = LRclf.predict(X_test)
print(f'Training Score: {LRclf.score(X_train, y_train)}')
print(f'Testing Score: {LRclf.score(X_test, y_test)}')
print(classification_report(y_test, y_pred))
balanced_accuracy_score(y_test, y_pred)
# cm = confusion_matrix(y_test, y_pred)
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
# cm_df


In [ ]:
from sklearn.tree import DecisionTreeClassifier
## DecisionTreeClassifier
DTCclf = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = DTCclf.predict(X_test)
print(f'Training Score: {DTCclf.score(X_train, y_train)}')
print(f'Testing Score: {DTCclf.score(X_test, y_test)}')
print(classification_report(y_test, y_pred)) 
print(balanced_accuracy_score(y_test, y_pred))

# cm = confusion_matrix(y_test, y_pred)
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
# cm_df



In [ ]:
## RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
RFCclf = RandomForestClassifier(random_state=1, n_estimators=3000).fit(X_train, y_train)
y_pred = RFCclf.predict(X_test)
print(f'Training Score: {RFCclf.score(X_train, y_train)}')
print(f'Testing Score: {RFCclf.score(X_test, y_test)}')
print(classification_report(y_test, y_pred)) 
print(balanced_accuracy_score(y_test, y_pred))

# cm = confusion_matrix(y_test, y_pred)
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
# cm_df




In [ ]:
## ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier
ETCclf = ExtraTreesClassifier(random_state=1, n_estimators=3000).fit(X_train, y_train)
y_pred = ETCclf.predict(X_test)
print(f'Training Score: {ETCclf.score(X_train, y_train)}')
print(f'Testing Score: {ETCclf.score(X_test, y_test)}')
print(classification_report(y_test, y_pred))
print(balanced_accuracy_score(y_test, y_pred))

# cm = confusion_matrix(y_test, y_pred)
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
# cm_df

